In [1]:
%cd ..
import pickle

/home/lingze/embedding_fusion


In [2]:
# first read the data
file_path = './tmp/studies_docs.pkl'
with open(file_path, 'rb') as f:
    studies_docs = pickle.load(f)

In [3]:
# construct the BM25 index
import bm25s

retriever = bm25s.BM25(backend="numba")
retriever.index(studies_docs)
retriever.activate_numba_scorer()

/home/lingze/anaconda3/envs/deepdb/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
from tqdm import tqdm
top_k = 101 # exclude the query itself
batch_size = 512

idx_to_positive_samples = []

for batch_idx in tqdm(range(0, len(studies_docs), batch_size)):
    batch_docs = studies_docs[batch_idx:batch_idx+batch_size]
    related_docs, _ = retriever.retrieve(batch_docs, k = top_k, n_threads = -1)
    related_docs = related_docs[:,1:] # exclude the query itself
    idx_to_positive_samples.append(related_docs)


In [8]:
import numpy as np
idx_to_positive_samples = np.concatenate(idx_to_positive_samples, axis = 0)

In [10]:
# save the reuslts
path = './tmp/studies_positive_samples.npy'
np.save(path, idx_to_positive_samples)